<a href="https://colab.research.google.com/github/machiwao/CCDEPLRL_PROJECT_COM222/blob/blix/CCDEPLRL_PROJECT_COM222_Project_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArtDecode: An Explainable Deep Learning-Based Mobile Application for Multi-Style Artistic Image Classification and Visual Feature Interpretation

In [1]:
import numpy as np
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from PIL import Image  # PIL is used to load the image
import matplotlib.pyplot as plt
import cv2

## Data Loading

Mount Google Drive Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load images and respective labels

In [3]:
base_dir = '/content/drive/My Drive/Primary Data' # Adjust this path if "Primary Data" is in a different location

In [4]:
styles = os.listdir(base_dir)
print(styles) # Print the list of styles to verify

['Primitivism', 'Art_Nouveau', 'Realism', 'Neoclassicism', 'Baroque', 'Japanese_Art', 'Academic_Art', 'Renaissance', 'Rococo', 'Expressionism', 'Romanticism', 'Symbolism', 'Western Medieval']


In [6]:
images = []
labels = []
for style in styles:
    style_dir = os.path.join(base_dir, style)
    for image_name in os.listdir(style_dir):
        image_path = os.path.join(style_dir, image_name)
        try:
            img = Image.open(image_path).convert('RGB') # Load image and convert to RGB
            img = img.resize((128, 128)) # Resize images to a consistent size
            img_array = np.array(img)
            images.append(img_array)
            labels.append(style) # Use the style name as the label
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

images = np.array(images)

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (96714256 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


## Data Split

In [33]:
# Train-Validation-Test Split is 70-10-20
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

In [34]:
# Convert y_train to a NumPy array to use NumPy functions
y_train_np = np.array(y_train)

# Get unique values and their counts
unique_elements, counts = np.unique(y_train_np, return_counts=True)

# Print the counts
print("Counts of each style in y_train:")
for element, count in zip(unique_elements, counts):
    print(f"{element}: {count}")

Counts of each style in y_train:
Academic_Art: 704
Art_Nouveau: 697
Baroque: 697
Expressionism: 701
Japanese_Art: 688
Neoclassicism: 687
Primitivism: 707
Realism: 688
Renaissance: 703
Rococo: 682
Romanticism: 697
Symbolism: 731
Western Medieval: 718


In [35]:
# Convert y_val to a NumPy array to use NumPy functions
y_val_np = np.array(y_val)

# Get unique values and their counts
unique_elements, counts = np.unique(y_val_np, return_counts=True)

# Print the counts
print("Counts of each style in y_val:")
for element, count in zip(unique_elements, counts):
    print(f"{element}: {count}")

Counts of each style in y_val:
Academic_Art: 113
Art_Nouveau: 111
Baroque: 98
Expressionism: 102
Japanese_Art: 88
Neoclassicism: 119
Primitivism: 93
Realism: 105
Renaissance: 102
Rococo: 110
Romanticism: 115
Symbolism: 96
Western Medieval: 74


In [36]:
label_to_index = {style: i for i, style in enumerate(styles)}
y_train_encoded = to_categorical([label_to_index[label] for label in y_train], num_classes=len(styles))
y_val_encoded = to_categorical([label_to_index[label] for label in y_val], num_classes=len(styles))
y_test_encoded = to_categorical([label_to_index[label] for label in y_test], num_classes=len(styles))

# Normalize the image data after splitting
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

## Model Training

In [37]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation="relu", input_shape=(128, 128, 3)), # Add input_shape here
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(256, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(styles))  # Output layer with number of classes
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    12,845,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 13)             │         3,341 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,219,469 (50.43 MB)

 Trainable params: 13,219,469 (50.43 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Initialize starting learning rate
initial_learning_rate = 0.0001
# Initialize optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
# Compile the model
model.compile(
    optimizer=optimizer,
    # Change the loss function to CategoricalCrossentropy for one-hot encoded labels
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing = 0.1),
    metrics=['accuracy']
)

In [43]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience = 20,
    restore_best_weights=True,
    verbose=1,
    min_delta=0.0001,
)

In [44]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,        # Reduce LR by half
    patience=10,        # Wait 3 epochs before reducing
    min_lr=1e-7,       # Don't go below this
    min_delta=0.0001,
    verbose=1
)

In [46]:
# Fit the model
history = model.fit(
    X_train, y_train_encoded, # Use the one-hot encoded training labels
    epochs=100,
    validation_data=(X_val, y_val_encoded), # Use the one-hot encoded validation labels
    callbacks=[early_stop, reduce_lr],
    batch_size=32
)

Epoch 1/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - accuracy: 0.1356 - loss: 2.4863 - val_accuracy: 0.2036 - val_loss: 2.3526 - learning_rate: 1.0000e-04
Epoch 2/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step - accuracy: 0.2919 - loss: 2.1466 - val_accuracy: 0.3137 - val_loss: 2.1013 - learning_rate: 1.0000e-04
Epoch 3/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.3508 - loss: 1.9723 - val_accuracy: 0.3341 - val_loss: 2.0402 - learning_rate: 1.0000e-04
Epoch 4/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.4147 - loss: 1.8120 - val_accuracy: 0.3718 - val_loss: 1.9521 - learning_rate: 1.0000e-04
Epoch 5/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.4488 - loss: 1.6927 - val_accuracy: 0.3620 - val_loss: 1.9570 - learning_rate: 1.0000e-04
Epoch 6/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.4924 - loss: 1.5527 - val_accuracy: 0.3763 - val_loss: 1.8838 - learning_rate: 1.0000e-04
Epoch 7/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 20s 3

## Model Evaluation

In [48]:
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 12))
  ax1.plot(history.history['accuracy'])
  ax1.plot(history.history['val_accuracy'])
  ax1.set_title('model accuracy')
  ax1.set_ylabel('accuracy')
  ax1.set_xlabel('epoch')
  ax1.legend(['train', 'validation'], loc='upper left')
  ax2.plot(history.history['loss'])
  ax2.plot(history.history['val_loss'])
  ax2.set_title('model loss')
  ax2.set_ylabel('loss')
  ax2.set_xlabel('epoch')

In [50]:
model.evaluate(X_test, y_test_encoded)

81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4329 - loss: 1.8061


[1.8297455310821533, 0.4176379144191742]

In [55]:
# Print Classification Report. Print which number represents which art style
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test_classes, y_pred_classes, target_names=styles))

81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
                  precision    recall  f1-score   support

     Primitivism       0.46      0.38      0.41       200
     Art_Nouveau       0.54      0.47      0.50       192
         Realism       0.36      0.37      0.37       207
   Neoclassicism       0.58      0.57      0.58       194
         Baroque       0.40      0.29      0.34       205
    Japanese_Art       0.72      0.65      0.69       224
    Academic_Art       0.43      0.46      0.45       183
     Renaissance       0.32      0.31      0.32       195
          Rococo       0.53      0.40      0.46       208
   Expressionism       0.23      0.48      0.31       197
     Romanticism       0.36      0.34      0.35       188
       Symbolism       0.20      0.22      0.21       173
Western Medieval       0.54      0.44      0.48       208

        accuracy                           0.42      2574
       macro avg       0.44      0.41      0.42      2574
    weighted avg       0.44   

In [54]:
# Print which number represents which art style
label_to_index = {style: i for i, style in enumerate(styles)}
index_to_label = {i: style for style, i in label_to_index.items()}
print(index_to_label)

{0: 'Primitivism', 1: 'Art_Nouveau', 2: 'Realism', 3: 'Neoclassicism', 4: 'Baroque', 5: 'Japanese_Art', 6: 'Academic_Art', 7: 'Renaissance', 8: 'Rococo', 9: 'Expressionism', 10: 'Romanticism', 11: 'Symbolism', 12: 'Western Medieval'}
